### import

In [1]:
import os, re
from os.path import join
import pandas as pd
import numpy as np
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import PatternFill, Border, Side, Font, Alignment
from openpyxl.utils import get_column_letter
basic_dtype = {'채무자키':str, '계좌키':str}

### 인적속성 - 다중채무, 다중담당 체크하기
- 주민번호기준으로 간단하게 채무상태 병합한 것은 "전산_자산정리_솔림.ipynb"에 있음

In [3]:
account = pd.read_pickle(r"D:\3.자산\전산 dataset\230430\계좌조회새창_20230502_0847.pkl")
new_debtKey = pd.read_pickle(r"C:\Users\SL\Desktop\workspace\python\fileNaming\파일\새채무자키.pkl")[["새채무자키","채무자키","계좌키"]]
# 새채무자키 병합하기
account.drop(account[account.채무자명=="예수금"].index, inplace=True)
account = pd.merge(account, new_debtKey[["계좌키","새채무자키",]], on='계좌키', how='left')

# 필요열만 새로운 df로 & 정렬
new_debtKey = account[["새채무자키", '채무자키','계좌키','채무상태','담당자']].copy()
new_debtKey.sort_values(['새채무자키', '채무자키', '계좌키'], inplace=True)

###### multi_index : 조작하기는 어렵고, 엑셀로 보낼때 자동으로 병합이 되어서 보기는 좋음
x = account.set_index(['새채무자키','채무자키','계좌키'],drop=True)["채무상태"]
# 다중채무, 다중담당 T/F값 입력
x = new_debtKey.groupby('새채무자키')
new_debtKey['다중채무상태여부'] = x['채무상태'].transform(lambda s: np.any(s != s.iloc[0]))
new_debtKey['다중담당여부'] = x['담당자'].transform(lambda s: np.any(s != s.iloc[0]))
# 다중인 것들만 
check = new_debtKey[new_debtKey["다중채무상태여부"] | new_debtKey['다중담당여부']].copy()
# index설정
gc = check.set_index(['새채무자키', '채무자키'], drop=True)
#저장
gc.to_excel(r"c://Users/SL/Desktop/check_채무상태(병합).xlsx")

### (새채무자키기준) 전체채무/일부채무

In [12]:
# 파일 읽기
####################################################################
# 최신/기준일 계좌전체(채무상태나 담당자 확인해야함)
path_account = r"D:\3.자산\전산 dataset\230430\계좌조회새창_20230502_0847" + ".pkl" 
# 확인하려는 풀의 채무자키와 계좌키, 저장경로
wd = r'c:/Users/SL/Desktop'
filename = 'POOL8-B'
filename_save = filename + "_채무자분리여부.xlsx"
####################################################################
new_debtkeys = pd.read_pickle(r"C:\Users\SL\Desktop\workspace\python\fileNaming\파일\새채무자키.pkl")[["새채무자키","채무자키","계좌키"]] # 계좌수만큼 있음
all_account = pd.read_pickle(path_account)[["채무자키","계좌키","종결일","채무상태","담당자","채무자명","현재원금","현재법비용","현재미수이자","현재연체이자"]].fillna("") # 엑셀 읽을땐 dtype = basic_dtype 
work_df = pd.read_excel(join(wd,filename+".xlsx"), dtype=basic_dtype)
# 혹시 풀 다른지 확인하기 위해
pool_info = pd.read_excel(r'D:\3.자산\POOL별 관리자산\전체\0.전체정리_230331_채권구분&POOL표시.xlsx', sheet_name="POOL 채권구분 채권상태 정리", header=2, dtype=basic_dtype)[["계좌키","채무자키","소속풀"]]

print(len(new_debtkeys), len(all_account), len(work_df), len(pool_info)) # 예수금 때문에 new_debtkeys와 all_account는 1개 차이남

78157 78158 1890


In [14]:
# work_all_account : 새채무자키파일에서 작업파일과 같은 동일한 사람들만 남기기
work_new_debtkeys = new_debtkeys[new_debtkeys.채무자키.isin(work_df.채무자키)].새채무자키
work_all_account = new_debtkeys[new_debtkeys.새채무자키.isin(work_new_debtkeys)].copy() # 새, 채무, 계좌 3개 행
# 매각대상 리스트에 포함된 계좌인지 여부 표시하기
work_all_account["포함여부"] = np.where(work_all_account.계좌키.isin(work_df.계좌키),"Y","N")
# 정렬
work_all_account.sort_values(by='새채무자키', inplace=True)
# result_new_debtkey : (sheet1용)채무상태, 담당자, 소속풀, 현재잔액 추가히기
result_new_debtkey = pd.merge(work_all_account, all_account[['계좌키','채무상태','담당자', '채무자명',"현재원금","현재법비용","현재미수이자","현재연체이자"]], on='계좌키', how="left")
result_new_debtkey = pd.merge(result_new_debtkey, pool_info[['계좌키',"소속풀"]], on='계좌키', how="left")

In [15]:
# result_debtkey : (sheet2용)채무자키 기준(전체계좌/일부계좌) - 전산 채무상태 및 담당자 변경 참고를 위함
result_debtkey = pd.DataFrame(None, columns=["채무자키", "전계좌여부", "채무상태", "담당자", "채무자명"])
temp = result_new_debtkey[result_new_debtkey.포함여부=="Y"].drop_duplicates('채무자키').copy()
result_debtkey["채무자키"] = temp.채무자키.values
result_debtkey["채무상태"] = temp.채무상태.values # 순서 조작만 하지 않으면 굳이 일치조건 하지 않아도 됨.
result_debtkey["담당자"] = temp.담당자.values
result_debtkey["채무자명"] = temp.채무자명.values

g = result_new_debtkey.groupby('채무자키') # 그룹화는 원본 df의 순서와 다를 수 있음!!!!!!!!
for i, v in result_debtkey.iterrows() : 
     if g.get_group(v.채무자키).포함여부.nunique() == 1 :
          result_debtkey.loc[i, "전계좌여부"]="전계좌"
     else : result_debtkey.loc[i, "전계좌여부"]="일부계좌"

In [21]:
###### multi_index : 조작하기는 어렵고, 엑셀로 보낼때 자동으로 병합이 되어서 보기는 좋음
result_new_debtkey = result_new_debtkey.set_index(['새채무자키','채무자키','계좌키'],drop=True)
result_debtkey = result_debtkey.set_index('채무자키',drop=True)

# 저장하기
with pd.ExcelWriter(join(wd,filename_save), engine='openpyxl', mode = 'w', ) as writer :
    result_new_debtkey.to_excel(writer, sheet_name="(새채무자키)동일인 분리여부")
    result_debtkey.to_excel(writer, sheet_name="(채무자키)다계좌 전체포함여부")

### (채무자키 기준)전체계좌 / 일부계좌 구분

In [4]:
# 파일 읽기
####################################################################
path_account = r"D:\3.자산\전산 dataset\230430\계좌조회새창_20230502_0847" + ".pkl" # 계좌전체. 채무상태나 담당자 변경하려면 최신파일로
path_work_df = r'c:/Users/SL/Desktop/통합 문서1.xlsx' # 확인하려는 풀의 채무자키와 계좌키
path_save = r'c:/Users/SL/Desktop/전체계좌여부.xlsx'
####################################################################
all_account = pd.read_pickle(path_account)[["채무자키","계좌키","종결일","채무상태","담당자","채무자명"]].fillna("") # 계좌조회
work_df = pd.read_excel(path_work_df, dtype={"채무자키":str,"계좌키":str})
print(len(all_account), len(work_df))

78158 453


In [5]:
# 채무자키로 전 계좌 불러오기
# 계좌시트에서 작업대상 채무자키에 해당하는 계좌 모두 불러오기
work_all_account = all_account[all_account.채무자키.isin(work_df.채무자키)].copy() # 객체 새로 만들지 않으면 계속 copywarning 나옴
# 불러온 계좌 중 작업 계좌에 있거나, 종결된 계좌면 Y, 아니면 N(솔림 잔존계좌)
cond = (work_all_account.계좌키.isin(work_df.계좌키)) | (work_all_account.종결일 != "")
work_all_account["계좌작업대상여부"] = np.where(cond, "Y", "N")

# 채무자키별 부수정보 담은 df 만들기
result = pd.DataFrame(None, columns=["채무자키", "전계좌여부", "채무상태", "담당자", "채무자명"])
temp = work_all_account.drop_duplicates('채무자키').copy()
result["채무자키"] = temp.채무자키.values
result["채무상태"] = temp.채무상태.values # 순서 조작만 하지 않으면 굳이 일치조건 하지 않아도 됨.
result["담당자"] = temp.담당자.values
result["채무자명"] = temp.채무자명.values


# 작업대상여부의 유니크값이 1이면 모두 Y (모두 N인 경우는 논리적으로 불가능), 2면 작업계좌와 잔존계좌가 혼재한 것이므로 일부계좌
g = work_all_account.groupby('채무자키') # 그룹화는 원본 df의 순서와 다를 수 있음!!!!!!!!
for i, v in result.iterrows() : 
     if g.get_group(v.채무자키).계좌작업대상여부.nunique() == 1 :
          result.loc[i, "전계좌여부"]="전계좌"
     else : result.loc[i, "전계좌여부"]="일부계좌"

# 저장하기
result.to_excel(path_save, index=False)

In [26]:
df = pd.DataFrame({"가":["삼", ""]})
df.가.nunique()

2

### 채무자 분리 전수조사

In [4]:
path_criterion = r"D:\3.자산\전산 정리\check\채무자분리\0.전수조사_기준파일.xlsx"
df_all = pd.read_excel(path_criterion, sheet_name="전체", dtype=basic_dtype).fillna("")
path_save = r"D:\3.자산\전산 정리\check\채무자분리"
basic_filename = "채무자분리_전수조사_result.xlsx"
filename = basic_filename

#### 일부만 하는 경우

In [15]:
# 파일읽기
#####################
sheet_name = "POOL1"
#####################
filename = sheet_name + "_" + basic_filename
df_part = pd.read_excel(path_criterion, sheet_name=sheet_name, dtype=basic_dtype).fillna("")
# 새채무자키 중복제거 for merge
df_unique = df_part['새채무자키'].drop_duplicates()
# 새채무자키를 통해 매각 대상이 아닌 채무도 불러오기
df = pd.merge(df_unique, df_all, on='새채무자키', how='left')
print(f'차이 : {len(df)} - {len(df_part)} = {len(df) - len(df_part)}')

차이 : 474 - 453 = 21


#### 전체

In [11]:
df = df_all.copy()
filename = basic_filename

#### 공통

In [16]:
# 순서유지를 위해 정렬
df.sort_values('새채무자키', ascending=True, inplace=True)
g = df.groupby('새채무자키') 

분리매각여부 = []
소속풀분리여부 = []

# 그룹화된 데이터프레임에 대해 반복문 수행
for _, group_df in g:
    # 그룹 내 채무자키의 고유값 개수 확인
    매각count = group_df["양수자"].nunique()
    풀count = group_df["소속풀"].nunique()
    
    # 분리매각여부 확인 후 리스트에 추가
    if 매각count >= 2:
        분리매각여부.extend(["Y"] * len(group_df))
    else:
        분리매각여부.extend(["N"] * len(group_df))

    # 소속풀 분리여부 확인 후 리스트에 추가
    if 풀count >= 2:
        소속풀분리여부.extend(["Y"] * len(group_df))
    else:
        소속풀분리여부.extend(["N"] * len(group_df))
# 컬럼 추가
df["채무자 분리매각여부"] = 분리매각여부
df["채무자 소속풀 분리여부"] = 소속풀분리여부

In [17]:
# 엑셀 쓰기
wb = Workbook()
ws = wb.active
ws.title = "채무자 분리여부"

# 데이터 쓰기
for row in dataframe_to_rows(df, index=False) : 
    ws.append(row)

# 스타일 지정
fill_gray = PatternFill(fill_type='solid', start_color="D0CECE", end_color="D0CECE")
fill_col = PatternFill(fill_type='solid', start_color='FF0072C6', end_color='FF0072C6')
font_col = Font( color='FFFFFF', bold=True)
divider = Border(top=Side(style='thin'))
alignment_v = Alignment(vertical='center')
alignment_h = Alignment(horizontal='center')

# 서식 쓰기(위 반복문 안에서 해도 되지만, for cell in row의 cell이 셀주소가 아니라 셀값이라서 
# for cell in ws[ws.max_row -1] 이런식으로 바꿔줘야 한다. 가독성을 위해 따로 쓰는게 낫다.
# 보통은 셀 병합을 먼저해주어야 서식이 유지된다.

# 컬럼명 스타일 적용
for cell in ws[1] : 
    cell.fill = fill_col
    cell.font = font_col

prev_key = None
for row in ws.iter_rows(min_row=2) : # 2행부터 시작
    # 공통적용
    for cell in row : cell.alignment = alignment_v
    
    current_key = row[0].value
    # 인덱스(새채무자키)값이 달라지면 실선 넣기
    if current_key != prev_key :
        for cell in row :
            cell.border = divider
        prev_key = current_key

    # 채무상태와 담당자에 따라 배경색 넣기
    if row[5].value == '환매' or row[6].value in ('종결', '환매') :
        for cell in row : cell.fill = fill_gray
        if row[5].value == '환매' : row[5].font = Font(bold=True)
        if row[6].value in ('종결', '환매') : row[6].font = Font(bold=True)

# 가운데 정렬
columns = ['D', 'E', 'H', 'I']
for column in columns :
    for cell in ws[column] :
        cell.alignment = alignment_h

# 틀고정 및 필터
ws.freeze_panes = "A2"
ws.auto_filter.ref = ws.dimensions


prev_new_debtor_key = None
prev_debtor_key = None
s_n, s_d = None, None
for row in ws.iter_rows(min_row=2):
    # "새채무자키" 열 병합
    new_debtor_key = row[0].value # n1000001

    if new_debtor_key == prev_new_debtor_key:
        if s_n == None : 
            s_n = row[0].coordinate[0] + str(row[0].row-1) # None일때(처음 같은 값 나왔을 때)만 지정
        e_n = row[0].coordinate # 같은 키가 나올때마다 갱신
    else : 
        if s_n != None  : # 지정값이 있고, 이제 새로운 키가 나왔을 때 
            ws.merge_cells(f'{s_n}:{e_n}')
            s_n = None # 병합후에는 s_n 초기화
        # 지정값 없는 경우(병합할 거 없는 경우) pass
        
    prev_new_debtor_key = new_debtor_key

    # "채무자키" 열 병합
    debtor_key = row[1].value

    if debtor_key == prev_debtor_key:
        if s_d == None : 
            s_d = row[1].coordinate[0] + str(row[1].row-1) # None일때(처음 같은 값 나왔을 때)만 지정
        e_d = row[1].coordinate # 같은 키가 나올때마다 갱신
    else : 
        if s_d != None  : # 지정값이 있고, 이제 새로운 키가 나왔을 때 
            ws.merge_cells(f"{s_d}:{e_d}")
            s_d = None # 병합후에는 s_d 초기화
        # 지정값 없는 경우(병합할 거 없는 경우) pass

    prev_debtor_key = debtor_key

if s_n != None : ws.merge_cells(f'{s_n}:{e_n}')
if s_d != None : ws.merge_cells(f"{s_d}:{e_d}")

# 저장 #####################################################################
wb.save(join(path_save, filename))

In [128]:
# 셀 서식 없이 멀티인덱스(병합)만 하고 저장
df_multi = df.set_index(['새채무자키','채무자키','계좌키'],drop=True).copy()
df_multi.to_excel(join(path_save, filename))